## Example: Working with geodatasets

In [ ]:
from hydromt import DataCatalog
from hydromt.utils import setuplog

logger = setuplog("raster data", log_level=10)
dc = DataCatalog(logger=logger, data_libs=["artifact_data=v0.0.8"])

Here, we illustrate some common GIS problems and how the functionality of the DataArray/Dataset vector accessor can be used. The data is accessed using the HydroMT [DataCatalog](https://deltares.github.io/hydromt/latest/api.html#data-catalog). For more information see the [Reading vector data](https://deltares.github.io/hydromt/latest/_examples/reading_vector_data.html) example.

### Geospatial attributes 

Some of the available geospatial attributes are listed here below. For all of attributes (and methods for that matter): check the [HydroMT API reference](https://deltares.github.io/hydromt/latest/api.html)

In [ ]:
# Get the waterlevel dataset from the datacatalog
# The waterlevels are series per location (points; here in lat lon)
ds = dc.get_geodataset("gtsmv3_eu_era5")

In [ ]:
# Coordinate reference system
ds.vector.crs

In [ ]:
# Geospatial data as geometry objects (we show the first 5 points)
ds.vector.geometry.head()

In [ ]:
# names of x- and y coordinates

(ds.vector.x_name, ds.vector.y_name)

In [ ]:
# Bounding box of the geospatial data
ds.vector.bounds

### Reprojection

In [ ]:
# Reproject data to Pseudo Mercator (EPSG: 3857)
ds_pm = ds.vector.to_crs(3857)

# Coordinate reference system
ds_pm.vector.crs

In [ ]:
# Check that the coordinate values are indeed no longer in degrees
ds_pm.lon.values[0]

### Conversion

In [ ]:
# Create an ogr compliant dataset from ds
# When written in netcdf4 format, this can be read by ogr (osgeo; QGIS)
from numpy import mean

ds_ogr = ds.vector.ogr_compliant(reducer=mean)
ds_ogr

In [ ]:
# Convert the geospatial data to wkt strings
ds_wkt = ds.vector.to_wkt()
ds_wkt

In [ ]:
# Convert the geospatial data to geometry objects
ds_geom = ds.vector.to_geom()
ds_geom

### I/O (Internal and External)

In [ ]:
# Create a dummy GeoDataFrame for this example
import numpy as np
from geopandas import GeoDataFrame
from pyproj import CRS
from shapely.geometry import Point

from hydromt.gis import GeoDataArray

gdf = GeoDataFrame(
    [{"Loc": "I", "Stuff": 1}, {"Loc": "II", "Stuff": 2}],
    geometry=[Point(0, 0), Point(1, 1)],
    crs=CRS.from_epsg(4326),
)

ds_gdf = GeoDataArray.from_gdf(gdf, np.arange(gdf.index.size))
ds_gdf

In [ ]:
# Write the Dataset of the DataCatalog to a GeoDataFrame
# Waterlevel has besides stations a time dimension
# GeoDataFrames don't like vectors/ 2d array's, so if you want to keep the variable it can be reduced along the time dimension

gdf = ds.vector.to_gdf(reducer=mean)
gdf.head()

In [ ]:
# Write the Dataset to an ogr compliant netcdf
from os.path import join

ds.vector.to_netcdf(join("tmpdir", "ds_ogr.nc"), ogr_compliant=True)

# It is indeed ogr compliant as ogrinfo.exe is able to read it
!ogrinfo tmpdir/ds_ogr.nc